In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_fake=pd.read_csv("Fake.csv")
df_true=pd.read_csv("True.csv")

In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_marge = pd.concat([df_fake, df_true], axis =0 )

In [ ]:
df = df_marge.drop(["title", "subject","date"], axis = 1)

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df["text"] = df["text"].apply(wordopt)

In [ ]:
x = df["text"]
y = df["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
sc=LR.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * sc))
print(classification_report(y_test, pred_lr))

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)

    return print("\n\nLR Prediction: {} ".format(output_lable(pred_LR[0]),))


In [ ]:
news = str(input())
manual_testing(news)

In [ ]:
import pickle
pickle.dump(LR,open('model.pkl', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))
score = loaded_model.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

def model_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR1 = loaded_model.predict(new_xv_test)

    return print("\n\nModel Prediction: {} ".format(output_lable(pred_LR1[0]),))

In [ ]:
news = str(input())
model_testing(news)